## ○貪欲法＆動的計画法による複数ナップザック問題の近似解推定

In [1]:
import time
import cvxpy
import random
import numpy as np
import pandas as pd

In [2]:
# サンプルデータの読み込み
df_sample = pd.read_csv('sample_3*30.csv', sep='\t')

In [3]:
df_sample.head(3)

,id,price_0,price_1,price_2,rating_0,rating_1,rating_2
0,1,380,385,389,4.1,3.3,4.0
1,2,943,947,961,10.1,8.9,9.3
2,3,980,980,987,10.4,8.7,10.3


In [4]:
# clientの数。
client_num = int((len(df_sample.columns)-1)/2)

In [5]:
# 初めにクライアントが持っていた枠をランダムに決める（別のデータに対応できるかを見るため。）
sample = df_sample.sample(frac=1).reset_index(drop=True)
sample.head(3)

,id,price_0,price_1,price_2,rating_0,rating_1,rating_2
0,17,1093,1082,1105,10.5,10.6,11.7
1,13,607,627,608,6.1,7.0,5.1
2,20,417,422,418,3.5,5.9,3.1


In [6]:
# クライアト0, 1, 2の元々持っていた枠をランダムに決める。
length = len(sample) # データフレームの長さ
bps = [0 for i in range(client_num+1)] # Break Points
bp = 0
original_prices = [0 for i in range(client_num)] # 元々持っていた枠の価格和。
original_points = [0 for i in range(client_num)] # 元々持っていた枠の視聴率和。

for i in range(client_num):
    bp += length//client_num
    bps[i+1] = bp
    df_original = sample[bps[i]:bps[i+1]]
    original_prices[i] = sum(df_original['price_' + str(i)]) # 価格の和。
    original_points[i] = sum(df_original['rating_' + str(i)]) # 視聴率の和。

In [7]:
print(original_prices) # 元々持っていた枠の価格和。
print(original_points) # 元々持っていた枠の視聴率和。

[6541, 6157, 6329]
[63.0, 59.49999999999999, 65.2]


***
***

### 〜ここからナップザック問題（貪欲法のアプローチ）のアルゴリズム〜

In [8]:
# 必要な視聴率和(=元々持っていた視聴率和)
require_ratings = list(original_points) # 参照してしまうことを避けるため。
id(require_ratings) == id(original_points)

False

In [9]:
# value=「視聴率/号数（価格）」を求める。(これを比較して、貪欲に取り出す。)
for i in range(client_num):
    df_sample['value_'+str(i)] = df_sample['rating_'+str(i)]/df_sample['price_'+str(i)]

# valueができていることの確認。
df_sample.head(3)

,id,price_0,price_1,price_2,rating_0,rating_1,rating_2,value_0,value_1,value_2
0,1,380,385,389,4.1,3.3,4.0,0.010789,0.008571,0.010283
1,2,943,947,961,10.1,8.9,9.3,0.010710,0.009398,0.009677
2,3,980,980,987,10.4,8.7,10.3,0.010612,0.008878,0.010436


<b><font color='Red'>※valueは、正の数じゃなくても良い（単なる大きさ比較だから。）</font>
    <br>しかし、余ったものをナップザック問題で解くので、そこでは正の数でないといけない。（０の方が大きいから。）
    <br>とはいえ、価格、視聴率のどちらかが負の数になることは考えにくいため、意識するる必要はなさげ。</b>

In [10]:
def Greedy_optimizer(df, require_ratings, num):
    check_point=0 # 目的を達成した（元々の視聴率和を超えた）クライアントの数。
    require_points = list(require_ratings) # 元のリストが変わらないようにする。
    df = df.copy() # 元のデータフレームが変わらないようにする。
    
    value_col_names = ['value_'+str(i) for i in range(num)] # valueのカラム名（ここから目的を達成したクライアントを取り除いていく。）
    get_ids = dict() # 獲得した枠のidを格納する。
    for i in range(num):
        get_ids[i] = []
    
    while check_point < num:
        df['Max_value'] = df.loc[:, value_col_names].max(axis=1) # (残っている)クライアントの価値の中で最も高い値。
        df['Max_value_client'] = df.loc[:, value_col_names].idxmax(axis=1).apply(lambda x:x.replace('value_', '')) # 最も高い価値をつけているクライアントの名前。
        df = df.sort_values(by='Max_value', ascending=False).reset_index(drop=True) # その価値でソートする。
        
        for i in range(len(df)):
            name = int(df.iat[0, -1]) # 最も高い値をつけているクライアントの名前。
            require_points[name] -= df.at[0, 'rating_'+str(name)] # 必要な視聴率和からその枠の視聴率を引く。
            get_ids[name] += [df.iat[0, 0]] # 獲得した枠のidを記録する。
            df = df[1:].reset_index(drop=True) # dfを下っていく。
            if min(require_points) < 0: # どこかのクライアントが、目的を達成したら、一回終わり。
                break

        min_client = require_points.index(min(require_points)) # 目的を達成したクライアント名。（番号）
        require_points[min_client] = 0 # 目的を達成したら、0にする。（今後のため。）
        drop_col_name = ['price_'+str(min_client), 'rating_'+str(min_client), 'value_'+str(min_client)]
        value_col_names.remove(drop_col_name[-1]) # 目的を達成したクライアントは、除く。

        check_point += 1 # チェックポイント＝目的を達成したクライアントの数。
            
    return list(get_ids.values())

In [11]:
# 2重のリストをフラットにする関数(重複は残る！)
def Flatten_dual(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [12]:
from ortoolpy import knapsack

start = time.time() # プログラム開始時間

obtained_idses = Greedy_optimizer(df_sample, require_ratings, client_num) # 獲得した枠のid
obtained_prices = [0 for i in range(client_num)] # 獲得した枠の価格和。
obtained_points = [0 for i in range(client_num)] # 獲得した枠の視聴率和。
rate = 1.05 # どれだけ価格の超過分を許容するか。

for i in range(client_num):
    df_sample['judge'] = df_sample['id'].apply(lambda x:x in obtained_idses[i]) # 各クライアントが獲得した枠を判断するカラム。
    result = df_sample[df_sample['judge']] 
    obtained_prices[i] = sum(result['price_' + str(i)]) # 獲得した枠の合計額。
    obtained_points[i] = sum(result['rating_' + str(i)]) # 獲得した枠の合計視聴率。
    
all_get_ids = Flatten_dual(obtained_idses) # クライアントが獲得したアカウント全てを足し合わせたもの。
df_sample['judge'] = df_sample['id'].apply(lambda x:x not in all_get_ids) # 残った枠を判別するカラム。
remain_df = df_sample[df_sample['judge']].reset_index(drop=True) # 残ったデータフレーム。
capacities = list(np.array(original_prices) * rate - np.array(obtained_prices)) # 元々持っていた枠の合計価格(の1.05倍)から獲得した枠の合計価格を引く。

print('*'*20, '貪欲法によるナップザック問題の解法の結果', '*'*20)
print('交換前の合計価格　:', original_prices)
print('獲得枠の合計価格　:', obtained_prices)
print('交換前の合計視聴率:', original_points)
print('獲得枠の合計視聴率:', obtained_points)
print('残りキャパシティー:', capacities)
display(remain_df.head(3))
    
count = 1
while len(remain_df)>0:
    capacities = list(np.array(original_prices) * rate - np.array(obtained_prices)) # 元々持っていた枠の合計価格(の1.05倍)から獲得した枠の合計価格を引く。
    add_idses = [[] for i in range(client_num)] # 各ナップザック問題の結果、獲得した枠のidを格納する。
    add_ids = [0 for i in range(client_num)]
    
    for i in range(client_num):
        size = list(np.array(remain_df['price_' + str(i)]))
        weight = list(np.array(remain_df['rating_' + str(i)]))
        capacity = capacities[i]
        add_ids[i] = knapsack(size, weight, capacity)[1]
        
    ids_dict = dict()
    for key in range(len(remain_df)): # それぞれのindex
        values = [] # そのidの枠を指定したクライアントを格納するリスト。
        for j in range(client_num): # それぞれのクライアントごとに、
            if key in add_ids[j]: # そのid（のindex）があれば、
                values.append(j) # valuesにクライアントの名前が格納される。
        if values: # そのidを指定したクライアントがいれば、
            value = random.choice(values) # その中からクライアントをランダムに選ぶ。
            ids_dict[key] = value # idと合わせて格納される。
            
    if len(ids_dict)==0:
        rate += 0.05*(count)
        count += 1
                
    for i in range(client_num):
        add_idses[i] = [ids for ids, client in ids_dict.items() if client == i] # クライアントごとに、獲得したid
        # obtained_idses[i] += add_idses[i]
        obtained_idses[i] += remain_df.query('index in ' + str(add_idses[i]))['id'].values.tolist()
        obtained_prices[i] += sum(remain_df.query('index in ' + str(add_idses[i]))['price_' + str(i)])
        obtained_points[i] += sum(remain_df.query('index in ' + str(add_idses[i]))['rating_' + str(i)])

    extraction_ids = Flatten_dual(add_idses) # 今回獲得されたidの集合を、取り除く。
    remain_df = remain_df.query('index not in ' + str(extraction_ids)).reset_index(drop=True)
    
print('*'*20, '動的計画法によるナップザック問題の解法の結果', '*'*20)
print('交換前の合計価格　:', original_prices)
print('獲得枠の合計価格　:', obtained_prices)
print('交換前の合計視聴率:', original_points)
print('獲得枠の合計視聴率:', obtained_points)
display(remain_df.head(3))

end = time.time() # プログラム終了時間
print('*'*24, 'プログラム処理経過時間', round(end-start,5), '[sec]', '*'*24)

******************** 貪欲法によるナップザック問題の解法の結果 ********************
交換前の合計価格　: [6541, 6157, 6329]
獲得枠の合計価格　: [6074, 5870, 6232]
交換前の合計視聴率: [63.0, 59.49999999999999, 65.2]
獲得枠の合計視聴率: [64.1, 66.7, 67.4]
残りキャパシティー: [794.0500000000002, 594.8500000000004, 413.4500000000007]


,id,price_0,price_1,price_2,rating_0,rating_1,rating_2,value_0,value_1,value_2,judge
0,6,137,155,152,1.4,1.4,1.3,0.010219,0.009032,0.008553,True
1,26,657,637,675,6.7,6.2,6.8,0.010198,0.009733,0.010074,True


******************** 動的計画法によるナップザック問題の解法の結果 ********************
交換前の合計価格　: [6541, 6157, 6329]
獲得枠の合計価格　: [6731, 6025, 6232]
交換前の合計視聴率: [63.0, 59.49999999999999, 65.2]
獲得枠の合計視聴率: [70.8, 68.10000000000001, 67.4]


,id,price_0,price_1,price_2,rating_0,rating_1,rating_2,value_0,value_1,value_2,judge


************************ プログラム処理経過時間 0.20472 [sec] ************************


<b>ここからやらなければならないこと。
    1. 処理時間の測定（数を増やして）。
    2. 価格を超えてしまった時の対処法。（上の処理時間と要相談）
    3. プログラムの見直しと改善。
</b>